# 时间旅行（Time Travel
LangGraph 中的 **时间旅行（Time Travel）** 功能，这是一种高级状态管理机制，允许用户回溯到工作流的历史状态并重新执行后续步骤。以下是核心概念的中文解析：

---

### ⏳ **时间旅行功能的核心价值**
1. **问题场景**：
   - 传统聊天机器人是**线性对话**：用户输入 → AI 响应 → 结束
   - **实际需求**：
     - 用户想从**历史节点重新探索**不同结果（如："回到第三步尝试其他方案"）
     - 修复错误后**重新执行后续流程**（如：修改参数后重试计算）
     - 类似 Git 的版本回退（常见于自主编程 Agent）

2. **技术基础**：
   - **检查点（Checkpoints）**：LangGraph 自动保存每个步骤的完整状态
   - **持久化层**：所有历史状态存储在数据库（如 Redis/PostgreSQL）
   - **状态标识**：每个状态有唯一 ID（如 `thread_id + step_id`）

---

### 🔄 **时间旅行工作原理**
```mermaid
sequenceDiagram
    participant User
    participant LangGraph
    participant DB

    User->>LangGraph: 发起任务
    LangGraph->>DB: 保存状态(step1)
    LangGraph->>User: 响应A
    User->>LangGraph: 继续任务
    LangGraph->>DB: 保存状态(step2)
    LangGraph->>User: 响应B

    Note over User: 用户选择"回到step1"
    User->>LangGraph: 提交历史状态ID
    LangGraph->>DB: 加载step1状态
    LangGraph->>LangGraph: 从step1重新执行
    LangGraph->>User: 新响应A'
```

---

### 🧩 **关键实现要素**
1. **状态加载 API**：
   ```python
   # 加载历史状态
   old_state = checkpoint.get(thread_id="123", step_id=2)
   
   # 从该状态重新执行
   new_state = graph.invoke(
       input={"new_input": "修正后的参数"},
       config={"configurable": {"thread_id": "123", "step_id": 2}}  # 指定起点
   )
   ```

2. **分支探索**：
   - 同一历史状态可衍生**多个执行分支**
   - 每个分支独立存储（类似代码仓库的 Git branch）

3. **状态完整性**：
   - 回退时自动隔离原始状态（避免污染历史数据）
   - 新分支继承原状态的**工具调用历史/消息记忆**

---

### 🚀 **典型应用场景**
1. **决策回溯**  
   - *场景*：金融分析 Agent 推荐股票A，用户想验证股票B  
   - *操作*：回退到分析节点，修改股票代码重新执行

2. **错误修复**  
   - *场景*：编程 Agent 第5步代码报错  
   - *操作*：回退到第4步，修改代码后继续执行

3. **策略对比**  
   - *场景*：实验设计 Agent 生成方案A，用户想比较方案B  
   - *操作*：从方案生成节点创建两个分支并行执行

4. **教育场景**  
   - *场景*：数学辅导 Agent 解题到第3步，学生想尝试不同解法  
   - *操作*：回溯到第2步重新推导

---

### ⚠️ **实现注意事项**
1. **工具副作用**：
   - 避免重复执行有副作用的工具（如支付/写数据库）
   - 方案：标记工具为 `idempotent`（幂等）或添加回滚逻辑
   ```python
   @tool(idempotent=True)  # 声明幂等工具
   def get_stock_price(symbol: str):
       # 可安全重复执行
   ```

2. **状态存储优化**：
   - 大尺寸状态（如图像生成）需压缩或外部存储
   - 设置自动清理策略（保留最近 N 个版本）

3. **分支管理**：
   - 为分支添加语义标签（如 `branch: "alternative_solution"`）
   - 可视化工具展示状态树（类似 Git Graph）

---

### 💡 **总结**
LangGraph 的时间旅行功能通过：
1. **状态快照**：自动保存每一步的完整上下文  
2. **精确回溯**：通过 `thread_id + step_id` 定位历史状态  
3. **分支执行**：从任意节点衍生新路径  

实现了 **"可逆计算"** 能力，解决了 AI Agent 工作流的**关键痛点**：
- 探索不确定性时的试错成本  
- 错误传导导致的推倒重来  
- 多方案对比的重复执行  

> 开发者建议：在涉及**复杂决策链**（如科研分析、代码生成）的场景中优先集成此功能，为用户提供 **"撤销/重试/分支探索"** 的核心交互体验。

In [ ]:
import os
from langchain.chat_models import init_chat_model

# 设置自定义API配置
os.environ["QWEN_API_KEY"] = "You API Key"
os.environ["QWEN_API_BASE"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"

# 使用自定义配置
llm = init_chat_model(
    model="qwen-plus-latest",
    model_provider="openai",
    api_key=os.environ["QWEN_API_KEY"],
    base_url=os.environ["QWEN_API_BASE"]
)

In [2]:
import os
import getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")

TAVILY_API_KEY:  ········


In [3]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    return Command(update=state_update)


tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

I can help you with that. Could you provide me with more details on what specific aspects of LangGraph you're interested in? For example, are you looking for an overview of its features, tutorials on how to use it, or something else?


In [6]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like a great plan! LangGraph is well-suited for building agents as it allows you to define complex workflows with ease. Here are some key points and steps that can help guide your project of building an autonomous agent:

### **Key Concepts in LangGraph**
1. **Graph-Based Workflows**: LangGraph lets you model workflows as graphs, making it easier to visualize and manage the flow of data and execution.
2. **Modular Components**: You can create modular components (nodes) representing specific tasks or functions, which can be reused across different workflows.
3. **State Management**: It provides tools to manage state effectively, allowing agents to remember previous interactions and make decisions based on historical data.
4. **Integration with LLMs**

In [7]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  8 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
-----------------------------------------------

In [8]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f061241-0473-6e16-8006-fd937852f970'}}


In [9]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

I can help you with that. Could you provide me with more details on what specific aspects of LangGraph you're interested in? For example, are you looking for an overview of its features, tutorials on how to use it, or something else?
